# Calculate symmetries

In [ ]:
import libpolycubes
import numpy as np
import utils

In [ ]:
utils.getSymms('000404040404840000000000')

In [ ]:
utils.getRotations(2)

In [ ]:
utils.getReflections(2)

In [ ]:
import pandas as pd
import altair as alt
from analysePhenotype import readAnalysed
#df = readAnalysed('/mnt/extraspace/joakim/210918/seeded/2d/out_19652_analysed.ftr')
df = readAnalysed('/mnt/extraspace/joakim/refcalc/seeded/2d/out_18067_analysed.ftr')
nSampled = 1e8
nDim = 2
method = 'stochastic'
alt.data_transformers.disable_max_rows()

In [ ]:
pd.set_option('display.max_colwidth', None)
df.loc[df['size'] == 16]

In [ ]:
alt.Chart(df.loc[df['size'] == 16]).mark_bar().encode(
    alt.X('count()'),
    alt.Y(alt.repeat("row"), type='nominal'),
).repeat(
    row=['rotsymms', 'reflsymms', 'invsymms']
)

In [ ]:
alt.Chart(df).mark_bar().encode(
    alt.X('count()'),
    alt.Y(alt.repeat("row"), type='nominal'),
).repeat(
    row=['rotsymms', 'reflsymms', 'invsymms']
)

The `reflsymms==10` polyominoes are all straight lines:

In [ ]:
df.loc[df['reflsymms'] == 10]

In [ ]:
df.loc[(df['reflsymms'] == 1) & (df['rotsymms'] == 0)]

In [ ]:
df.loc[df['rotsymms'] == 3]

In [ ]:
utils.getSymms('85000088000000000a0c000005158e0000000000001000008d9592000000', ndim=2)

# Symmetry groups
https://www.conwaylife.com/wiki/Symmetry

https://symotter.org/tutorial/pointgroups

## D4 symmetry
Symmetric under both reflection and 180° rotation.

In [ ]:
df.loc[(df['rotsymms'] == 1) & (df['reflsymms'] > 0)].sort_values(by=['count'])

In [ ]:
df.loc[df['reflsymms'] == 2].sort_values(by=['count'])

## C4 symmetry
Symmetric under 90° rotation

In [ ]:
df.loc[df['rotsymms'] == 3]

## D2 symmetry
Symmetric under reflection through a line.

In [ ]:
df.loc[df['reflsymms'] > 0]

## C2 symmetry
Symmetric under 180° rotation

There's something odd going on here...

In [ ]:
df.loc[df['rotsymms'] == 1]

## D1 symmetry
No reflectional symmetry

In [ ]:
df.loc[df['reflsymms'] == 0]

## C1 symmetry
No rotational symmetry

In [ ]:
df.loc[df['rotsymms'] == 0]

# Assign groups

In [ ]:
def getSymmGroup(row):
    if row['reflsymms'] == 2:
      return 'D4'
    if row['rotsymms'] == 3:
      return 'C4'
    if row['reflsymms'] == 1:
      return 'D2'
    if row['rotsymms'] == 1:
      return 'C2'
    if row['reflsymms'] == 0:
      return 'D1'
    if row['rotsymms'] == 0:
      return 'C1'
    return 'Other'

In [ ]:
df['symmetry_group'] = df.apply(lambda row: getSymmGroup(row), axis=1)

In [ ]:
df

In [ ]:
alt.Chart(df).mark_bar().encode(
    alt.X('count()'),
    alt.Y('symmetry_group:N', title="Symmetry group"),
    alt.Color('symmetry_group:N', legend=None)
)

# Frequency of symmetry

In [ ]:
def plotFreqSym(path, nSampled, nDim, method, minCount=1, nMer = None, saveHTML=False):
    title = 'Frequency vs complexity, {:.1E} samples of {}D {}'.format(nSampled, nDim, method)
    tmp = "{}/{}/{}d".format(path, method, nDim)
    [filename] = !ls $tmp/*.ftr
    print(filename)
    df = readAnalysed(filename)
    df = df.loc[df['count'] >= minCount]
    df['symmetry_group'] = df.apply(lambda row: getSymmGroup(row), axis=1)
    df = df.sort_values(['count'], ascending=True)
    #df = df.sort_values(['symmetry_group'], ascending=True, key=lambda col: col.str[-1])
    if nMer is not None:
        df = df.loc[df['size'] == nMer]
        title = '{}-mer frequency vs complexity, {:.1E} samples of {}D {}'.format(nMer, nSampled, nDim, method)
    chart = alt.Chart(df).mark_circle(size=25, opacity=0.6).encode(
        alt.X(alt.repeat("column"), type='quantitative', axis=alt.Axis(tickMinStep = 1)),
        alt.Y('freq:Q', scale=alt.Scale(type='log'), title="Frequency"),
        href='url:N',
        color=alt.Color('symmetry_group:N'),
        tooltip=['url:N', 'count', 'freq:Q', 'minLz', 'minNc', 'minNt', 'rotsymms', 'reflsymms', 'invsymms']
    ).transform_calculate(
        freq='datum.count/{}'.format(nSampled),
        url='"https://akodiat.github.io/polycubes/?rule="+datum.minLz_r',
        symmetries='1+datum.rotsymms+datum.reflsymms+datum.invsymms'
    ).properties(
        width=200,
        height=200
    ).repeat(
        column=['minNt', 'minNc', 'minLz']
    ).properties(title=title)
    if saveHTML:
        chart.save('freq_vs_compl.html')
    return chart

In [ ]:
plotFreqSym(
    path = '/mnt/extraspace/joakim/refcalc',
    nSampled = 1e9, nDim = 2, method = 'seeded', minCount = 1
)

In [ ]:
plotFreqSym(
    path = '/mnt/extraspace/joakim/refcalc',
    nSampled = 1e9, nDim = 2, method = 'seeded', minCount = 1, nMer = 16
)

In [ ]:
def plotFreqSym_nC(path, nSampled, nDim, method, minCount=1, nMer = None, saveHTML=False):
    title = 'Frequency vs complexity, {:.1E} samples of {}D {}'.format(nSampled, nDim, method)
    tmp = "{}/{}/{}d".format(path, method, nDim)
    [filename] = !ls $tmp/*.ftr
    print(filename)
    df = readAnalysed(filename)
    df = df.loc[df['count'] >= minCount]
    df['symmetry_group'] = df.apply(lambda row: getSymmGroup(row), axis=1)
    df = df.sort_values(['count'], ascending=True)
    #df = df.sort_values(['symmetry_group'], ascending=True, key=lambda col: col.str[-1])
    if nMer is not None:
        df = df.loc[df['size'] == nMer]
        title = '{}-mer frequency vs complexity, {:.1E} samples of {}D {}'.format(nMer, nSampled, nDim, method)
    chart = alt.Chart(df).mark_circle(size=40).encode(
        alt.X('minNc:Q', axis=alt.Axis(tickMinStep = 1), title="Complexity (Number of colours)"),
        alt.Y('freq:Q', scale=alt.Scale(type='log'), title="Frequency"),
        href='url:N',
        color=alt.Color('symmetry_group:N'),
        tooltip=['url:N', 'count', 'freq:Q', 'minLz', 'minNc', 'minNt', 'rotsymms', 'reflsymms', 'invsymms']
    ).transform_calculate(
        freq='datum.count/{}'.format(nSampled),
        url='"https://akodiat.github.io/polycubes/?rule="+datum.minLz_r',
        symmetries='1+datum.rotsymms+datum.reflsymms+datum.invsymms'
    ).properties(
        width=300,
        height=200
    ).properties(title=title)
    if saveHTML:
        chart.save('freq_vs_compl.html')
    return chart

In [ ]:
plotFreqSym_nC(
    path = '/mnt/extraspace/joakim/refcalc',
    nSampled = 1e9, nDim = 2, method = 'seeded', minCount = 1, nMer = 16
)

In [ ]:
def plotRank(path, nSampled, nDim, method, minCount=1, nMer = None, saveHTML=False):
    title = 'Frequency vs rank, {:.1E} samples of {}D {}'.format(nSampled, nDim, method)
    tmp = "{}/{}/{}d".format(path, method, nDim)
    [filename] = !ls $tmp/*.ftr
    print(filename)
    df = readAnalysed(filename)
    df = df.loc[df['count'] >= minCount]
    if nMer is not None:
        df = df.loc[df['size'] == nMer]
        title = '{}-mer frequency vs rank, {:.1E} samples of {}D {}'.format(nMer, nSampled, nDim, method)
    chart = alt.Chart(df).mark_circle(size=60).encode(
        alt.X("rank:O",scale=alt.Scale(type='log')),
        alt.Y('symmetries:Q', scale=alt.Scale(type='log'), title="Symmetries"),
        href='url:N',
        color=alt.Color('symmetries:Q', scale=alt.Scale(scheme="inferno")),
        tooltip=['url:N', 'count', 'freq:Q', 'minLz', 'minNc', 'minNt'],
    ).transform_calculate(
        freq='datum.count/{}'.format(nSampled),
        url='"https://akodiat.github.io/polycubes/?rule="+datum.minLz_r',
        symmetries='1+datum.rotsymms+datum.reflsymms+datum.invsymms'
    ).properties(width=350, height=200
    ).transform_window(
        rank="rank()",
        sort=[alt.SortField("count", order="descending")]
    ).properties(title=title)
    if saveHTML:
        chart.save('freq_vs_rank.html')
    return chart

In [ ]:
plotRank(
    path = '/mnt/extraspace/joakim/210918',
    nSampled = 1e8, nDim = 2, method = 'seeded', minCount = 10
)